In [3]:
import numpy as np

In [4]:
def normalFluxJacobian2D(H,Hux,Huy,g,nx,ny):

    ## return the normal flux jacobian (2-D)

    a = np.sqrt(g*H)
    ux = Hux/H
    uy = Huy/H

    Fn = np.zeros( (3,3) )

    Fn[0,:] = ( 0., nx, ny )
    Fn[1,:] = ( (a**2 - ux**2)*nx - ux*uy*ny , 2.*ux*nx + uy*ny, ux*ny )
    Fn[2,:] = ( -ux*uy*nx + (a**2-uy**2)*ny, uy*nx, ux*nx + 2.*uy*ny )

    return Fn

In [5]:
def eigenDecomp1D(H,Hux,g):
    
    ## return the eigendecomposition of the flux jacobian (1-D)
    
    ux = Hux/H

    a = np.sqrt(H*g)
    
    lam = np.zeros(2)
    lam[0] = ux - a
    lam[1] = ux + a
    
    rightEV = np.zeros( (2,2) )
    leftEV  = np.zeros( (2,2) )
    
    rightEV[:,0] = ( 1., ux-a )
    rightEV[:,1] = ( 1., ux+a )
    
    leftEV[0,:] = ( ux+a, -1. )
    leftEV[1,:] = ( a-ux,  1. )
    leftEV /= 2.*a
    
    return lam, leftEV, rightEV

In [7]:
def eigenDecomp2D(H,Hux,Huy,g,nx,ny):
    
    ## return the eigendecomposition of the flux jacobian (2-D)
    
    ux = Hux/H
    uy = Huy/H
    vn = ux*nx + uy*ny

    a = np.sqrt(g*H)
    
    lam = np.zeros(3)
    
    lam[0] = vn + a
    lam[1] = vn
    lam[2] = vn - a
    
    rightEV = np.zeros( (3,3) )
    leftEV  = np.zeros( (3,3) )
    
    rightEV[:,0] = ( 1., ux + a*nx, uy + a*ny )
    rightEV[:,1] = ( 0., -a*ny, a*nx )
    rightEV[:,2] = ( 1., ux - a*nx, uy - a*ny )
    
    leftEV[0,:] = 1./(2.*a)*np.array( [ a-vn, nx, ny ] )
    leftEV[1,:] = 1./(2.*a)*np.array( [ 2*(ux*ny - uy*nx), -2.*ny, 2.*nx ] )
    leftEV[2,:] = 1./(2.*a)*np.array( [ a + vn, -nx, -ny ] )
    
    return lam, leftEV, rightEV
    

In [6]:
def generateInputOutputEigDecomp(ndim=1):
    
    rng = np.random.default_rng()
    
    H = rng.uniform(0.,20.)
    ux = rng.uniform(-10.,10.)
    uy = rng.uniform(-10.,10.)
    nx = 1.
    g = 9.81 
    
    Hux = H*ux
    Huy = H*uy

    Fn = None

    if ndim == 1:
        lam,LEV,REV = eigenDecomp1D(H,Hux,g)
    elif ndim == 2:
        nx = rng.uniform(-1.,1.)
        nyMag = np.sqrt(1. - nx**2)
        ny = nyMag * rng.choice([1.,-1.])
        lam,LEV,REV = eigenDecomp2D(H,Hux,Huy,g,nx,ny)
        Fn = normalFluxJacobian2D(H,Hux,Huy,g,nx,ny)
    
    ## Print out format so we can plop directly in code...
    
    print( f"AD Hux = {Hux}; AD H = {H}; \n" )
    
    if ndim == 2:
        print ( f"AD Huy = {Huy}; ScalarT nx = {nx}; ScalarT ny = {ny};\n" )
    
    lamstring = ""
    LEVstring = ""
    REVstring = ""
    
    for j in range(1+ndim):
        lamstring += f"lamExact({j}) = {lam[j]}; "
        for i in range(1+ndim):
            LEVstring += f"LEvExact({i},{j}) = {LEV[i,j]}; "
            REVstring += f"REvExact({i},{j}) = {REV[i,j]}; "
        LEVstring += "\n"
        REVstring += "\n"
    lamstring += "\n"
    
    print (lamstring)
    print (LEVstring)
    print (REVstring)

    ## check here that the eigendecomp works
    if Fn is not None:
        L = np.diag(lam)
        A = REV@L@LEV
        print(np.linalg.norm(A-Fn))

In [8]:
generateInputOutputEigDecomp(2)

AD Hux = -81.22763890561589; AD H = 11.67604853386832; 

AD Huy = 88.01900202809388; ScalarT nx = 0.3998509528315446; ScalarT ny = 0.9165801740817362;

lamExact(0) = 14.83032811881777; lamExact(1) = 4.12789680078315; lamExact(2) = -6.57453451725147; 

LEvExact(0,0) = 0.30715144633410313; LEvExact(1,0) = -0.8774349875363572; LEvExact(2,0) = 0.6928485536658969; 
LEvExact(0,1) = 0.018680379296513564; LEvExact(1,1) = -0.08564223837037954; LEvExact(2,1) = -0.018680379296513564; 
LEvExact(0,2) = 0.04282111918518977; LEvExact(1,2) = 0.03736075859302713; LEvExact(2,2) = -0.04282111918518977; 

REvExact(0,0) = 1.0; REvExact(1,0) = -2.677397328669729; REvExact(2,0) = 17.348060149484564; 
REvExact(0,1) = 0.0; REvExact(1,1) = -9.809636360581997; REvExact(2,1) = 4.279377360130306; 
REvExact(0,2) = 1.0; REvExact(1,2) = -11.236152048930341; REvExact(2,2) = -2.2712125716794302; 

2.018076651186654e-14


In [9]:
def stabTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,method='Roe-like'):

    ## form delta S

    S = np.array( [H,Hux,Huy] )
    Shat = np.array( [ Hhat,Huxhat,Huyhat] ) 

    deltaS = S - Shat

    ## Get eigendecomp

    Fnhat = normalFluxJacobian2D(Hhat,Huxhat,Huyhat,g,nx,ny)
    l,rv = np.linalg.eig(Fnhat)

    if method == 'max-EV':
        return np.abs(l).max()*deltaS

    ## form roe-like stabilization R | lambda | R^-1
    tau = rv@np.diag(np.abs(l))@np.linalg.inv(rv)    

    return tau@deltaS

In [10]:
def boundTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,type='far-field'):

    ## form delta S

    S = np.array( [H,Hux,Huy] )
    Shat = np.array( [ Hhat,Huxhat,Huyhat] ) 

    ## Get eigendecomp

    Fnhat = normalFluxJacobian2D(Hhat,Huxhat,Huyhat,g,nx,ny)
    l,rv = np.linalg.eig(Fnhat)

    ## get absolute value of A via eigendecomp
    absFnhat = rv@np.diag(np.abs(l))@np.linalg.inv(rv)

    Aplus = 1./2. * ( Fnhat + absFnhat )
    Aminus = 1./2. * ( Fnhat - absFnhat )

    ## TODO needs to match with code and maybe we should at least generate like the state...
    ## TODO get something the exercises the two limits? A = A^+, A = A^-?
    Sinfinity = np.array( [8.2641,-2.5930,3.8813] )

    return Aplus@S - absFnhat@Shat - Aminus@Sinfinity

In [11]:
def fluxes2D(H,Hux,Huy,g):

    Fx = np.array( [Hux,Hux*Hux/H + g*H*H/2.,Hux*Huy/H] )
    Fy = np.array( [Huy,Huy*Hux/H,Huy*Huy/H + g*H*H/2.] )

    return Fx,Fy

In [12]:
def generateInputOutputStabBoundAndFluxes(ndim=2):
    
    rng = np.random.default_rng()
    
    H = rng.uniform(0.,20.)
    ux = rng.uniform(-10.,10.)
    uy = rng.uniform(-10.,10.)
    nx = 1.
    g = 9.81 
    
    Hux = H*ux
    Huy = H*uy

    Hhat = rng.uniform(0.,20.)
    uxhat = rng.uniform(-10.,10.)
    uyhat = rng.uniform(-10.,10.)

    Huxhat = Hhat*uxhat
    Huyhat = Hhat*uyhat

    nx = rng.uniform(-1.,1.)
    nyMag = np.sqrt(1. - nx**2)
    ny = nyMag * rng.choice([1.,-1.])

    stab = stabTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,method='Roe-like')
    bound = boundTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,type='far-field')
    
    ## Print out format so we can plop directly in code...
    
    print( f"vector<AD> solvals = {{{H},{Hux},{Huy}}};" )
    print( f"vector<AD> auxvals = {{{Hhat},{Huxhat},{Huyhat}}};" )
    print( f"ScalarT nxVal = {nx}; ScalarT nyVal = {ny};" )
    
    print( f"vector<AD> stabExactVals = {{{stab[0]},{stab[1]},{stab[2]}}};" )

    stab = stabTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,method='max-EV')

    print ("\n Now max EV result...\n")
    print( f"stabExactVals = {{{stab[0]},{stab[1]},{stab[2]}}};" )

    print( "\n" )
    print( f"vector<AD> boundExactVals = {{{bound[0]},{bound[1]},{bound[2]}}};")

    Fx,Fy = fluxes2D(H,Hux,Huy,g)

    print ("\n")
    print( f"vector<AD> FxExactVals = {{{Fx[0]}, {Fx[1]}, {Fx[2]}}};")
    print( f"vector<AD> FyExactVals = {{{Fy[0]}, {Fy[1]}, {Fy[2]}}};")

    Fx,Fy = fluxes2D(Hhat,Huxhat,Huyhat,g)

    print("")
    print( f"FxExactVals = {{{Fx[0]}, {Fx[1]}, {Fx[2]}}};")
    print( f"FyExactVals = {{{Fy[0]}, {Fy[1]}, {Fy[2]}}};")

In [13]:
generateInputOutputStabBoundAndFluxes()

[ -165.40000438 -2124.5484996   2171.05709022]
[ 33.93091906 -85.68962398 311.63482572]
vector<AD> solvals = {14.90359295934681,-132.57586829074378,87.52517363958512};
vector<AD> auxvals = {18.516385131727183,73.43579650871726,-48.64133286248697};
ScalarT nxVal = 0.48164387059090896; ScalarT nyVal = -0.8763670360768984;
vector<AD> stabExactVals = {-112.24391542943832,-2258.789640156109,2659.263392186651};

 Now max EV result...

stabExactVals = {-63.91013085278527,-3644.337074019624,2408.779368723075};


vector<AD> boundExactVals = {-199.33092344365062,-2038.8588756230874,1859.4222645048887};


vector<AD> FxExactVals = {-132.57586829074378, 2268.8214431056745, -778.5858030488429};
vector<AD> FyExactVals = {87.52517363958512, -778.5858030488429, 1603.4983321315206};

FxExactVals = {73.43579650871726, 1972.95685010534, -192.9110350962538};
FyExactVals = {-48.64133286248697, -192.9110350962538, 1809.4888230202857};
